In [5]:
import transformers
print(transformers.__version__)

4.48.3


In [11]:
pip install evaluate

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: http://repo.ai.gato/registry/repository/pypi-proxy/simple
  Using cached http://repo.ai.gato/registry/repository/pypi-proxy/packages/fsspec/2024.9.0/fsspec-2024.9.0-py3-none-any.whl (179 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2025.2.0 requires fsspec==2025.2.0.*, but you have fsspec 2024.9.0 which is incompatible.
torch 2.6.0 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.13.3 which is incompatible.
torch 2.6.0 requires triton==3.2.0; platform_system == "Linux" and platform_machine == "x86_64", but you have triton 3.1.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import time
import torch
from datasets import load_dataset
import evaluate
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# 데이터셋 로딩
dataset = load_dataset("squad_kor_v1")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [23]:
dataset["train"][0]

{'id': '6566495-0-0',
 'title': '파우스트_서곡',
 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': {'text': ['교향곡'], 'answer_start': [54]}}

In [27]:
from datasets import load_dataset

def formatting_prompts_func(examples):
    eos_token = '<|end_of_text|>' # Llama 모델의 eos token
    korQuAD_prompt = """
    
    # 학습시킬 Prompt의 형태
        ### Question:
        {}

        ### Context:
        {}

        ### Answer:
        {}
    """
	
    #데이터셋에서 원하는 컬럼을 가져와 instruction, input, output으로 매핑
    instructions = examples["question"]
    inputs = examples["context"]
    outputs = [item['text'][0] for item in examples["answers"]]
    texts = []
	
    #데이터 뒤에 eos_token을붙여서 최종 데이터 가공 
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = korQuAD_prompt.format(instruction, input, output) + eos_token
        texts.append(text)

    return {"text": texts}


In [26]:
dataset = load_dataset("KorQuAD/squad_kor_v1", split = "train")
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,        
)
print(dataset[:1])

README.md:   0%|          | 0.00/6.29k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/11.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

Map:   0%|          | 0/60407 [00:00<?, ? examples/s]

{'id': ['6566495-0-0'], 'title': ['파우스트_서곡'], 'context': ['1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.'], 'question': ['바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'], 'answers': [{'text': ['교향곡'], 'answer_start': [54]}], 'text': ['\t\t  \t# 학습시킬 Prompt의 형태\n        ### Question:\n        바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?\n\n        ### Context:\n        1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내

In [21]:
import time
import torch
from transformers import TrainingArguments, Trainer, AutoModelForCausalLM

# 공통 학습 하이퍼파라미터 설정 (DLPC 12GB 환경 고려)
common_training_args = {
    "num_train_epochs": 3,
    "per_device_train_batch_size": 2,       # VRAM 제약 고려
    "per_device_eval_batch_size": 2,
    "gradient_accumulation_steps": 16,        # 유효 배치 사이즈 증가
    "eval_strategy": "epoch",
    "save_strategy": "epoch",
    "logging_steps": 50,
    "learning_rate": 2e-5,
    "fp16": True,
    "report_to": "none",
}

# Baseline 학습: QLoRA 미적용
baseline_args = TrainingArguments(output_dir="./results_baseline", **common_training_args)

# Baseline 모델 로드 (4-bit quantization 없이)
baseline_model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=False)
baseline_model.to("cuda")

# (필요 시, 토크나이저 변경에 따른 임베딩 크기 재조정)
baseline_model.resize_token_embeddings(len(tokenizer))

trainer_baseline = Trainer(
    model=baseline_model,
    args=baseline_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

# 학습 시간 및 VRAM 측정을 위한 초기화 후 학습 진행
torch.cuda.reset_peak_memory_stats()
start_time = time.time()
trainer_baseline.train()
baseline_training_time = time.time() - start_time
baseline_peak_memory = torch.cuda.max_memory_allocated("cuda") / (1024 ** 2)  # MB 단위

print("Baseline Training Time: {:.2f} sec".format(baseline_training_time))
print("Baseline Peak VRAM Usage: {:.2f} MB".format(baseline_peak_memory))


/tmp/ipykernel_19181/2739914875.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_baseline = Trainer(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


OutOfMemoryError: CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 0 has a total capacity of 23.68 GiB of which 236.75 MiB is free. Process 61661 has 28.06 MiB memory in use. Process 112229 has 23.42 GiB memory in use. Of the allocated memory 23.03 GiB is allocated by PyTorch, and 74.60 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)